<a href="https://colab.research.google.com/github/solharsh/Experimenting_ML/blob/master/Dont_Overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avoid overfitting with a tiny sliver for training data
Inspired by the Kaggle Don’t Overfit Challenge: Tiny Training Trial. The challenge; build the best performing model you can with a <5% training vs >95% test split with TF-IDF encodings on an Amazon multi-classification problem. With so many data hungry algorithms out there that take days or more to compute, we thought it’d be refreshing to go the other way and experiment with what can be done with extremely small and noisy datasets! Iterate and experiment with training times on the order of seconds. Our split is:

Train: 1244 points

Approach overview

•Build Ensemble that includes multiple model categories: Logistic Regression, Random Forests, XGBoost, Adaboost, and Neural Networks.

•Split the training dataset into K stratified folds. For each fold and model category, train a separate model using Grid Search.

•Combine all models into ensemble using Averaging.

I experimented with:

- 1.Which model categories to include in the ensemble 
- 2.How many stratified folds to use: 1, 5, 10, 20, 40 
- 3.How to build the ensemble: Averaging vs. Max voting
- 4.Oversampling techniques such as SMOTE and ADASYN: including models trained with SMOTE data in the ensemble worked for the Public leaderboad, but not for Private
- 5.Feature standardization: did not seem to improve anything.

## Lessons Learned

-  Ensembling is the way to go, of course.
-  Increasing the number of stratified folds improved performance.
-  Improvements in training data accuracy (on validation set) did not necessarrily translate to better accuracies in the Public dataset. A prime example for this was the LR method that did not perform as well in the training validation accuracy compared to other methods such as NN. However, LR was an integral part of the overall Ensemble; whenever we removed it, the Public dataset accuracy ended up much worse.
-  Ensembling using Averaging always worked better than Max voting.
-  We kind of `overfitted' to the Public Leaderboard, i.e., our best performing model in Public was not the best in Private. 
-  Adding models trained with oversampled data, using either SMOTE or ADASYN, decreased accuracy in Private dataset. 
-  Gini impurity appeared to work better than Entropy for tree-based models.

## Initialization

In [4]:
import pandas as pd, numpy as np, time, sys, h5py
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from keras.layers import Input, Dense , Dropout , TimeDistributed , LSTM , GRU, concatenate, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD , Adadelta, RMSprop, Adam, Adamax
from keras.models import  load_model
from keras.callbacks import EarlyStopping
from keras.utils import  to_categorical 
from keras.regularizers import l1, l2, l1_l2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pickle
from sklearn.svm import SVC

Using TensorFlow backend.


In [0]:
# Initialize problem parameters
class Args:
    """ Class containing all model arguments """
    def __init__( self ):
        self.project    = 'MLchallenge_DontOverfit'
        self.dataPath   = '/content/drive/My Drive/Dont_Overfit/'       .format(self.project)
        self.modelsPath = '/content/drive/My Drive/Dont_Overfit/' .format(self.project)
        self.resultsPath= '/content/drive/My Drive/Dont_Overfit/'.format(self.project)
        self.CV_folds   = 40  # split the Training data in stratified folds, to train different versions of models 
args = Args()

In [6]:
# LOAD DATA
train = pd.read_csv( args.dataPath + 'TTT_train.csv' )
test  = pd.read_csv( args.dataPath + 'TTT_test_features.csv',index_col = 'ID')
train.describe()
X = train.loc[:, train.columns != 'label']
y = train['label']
y_cat = to_categorical(y)
# Generate a set of stratified folds of Training to train different versions of each model.
folds = list(StratifiedKFold(n_splits=args.CV_folds, shuffle=True, random_state=1).split(X, y))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 36 members, which is less than n_splits=40.
  % (min_groups, self.n_splits)), UserWarning)


## Some functions for model training and validation

In [0]:
##################################################
# function to fit a model on every fold, and store trained model
def fitValidateSave( model, modelType ):
    #
    accuracies = []
    for foldIndex, fold in enumerate(folds):
        X_fold      = np.take( X, fold[0], axis=0)
        y_fold      = np.take( y, fold[0], axis=0)
        #
        #oversampler = RandomOverSampler(random_state=77)
        #X_fold, y_fold = oversampler.fit_sample(X_fold, y_fold)
        #
        X_fold_test = np.take( X, fold[1], axis=0)
        y_fold_test = np.take( y, fold[1], axis=0)
        #
        model.fit(X_fold, y_fold)
        #
        accuracies.append( model.score(X_fold_test, y_fold_test) )
        print( '{}: {}'.format(foldIndex, accuracies[-1]) )
        print(model.best_params_)
        #
        pickle.dump( model, open( '{}/{}_fold{}.h5'.format( args.modelsPath, modelType, foldIndex ) , 'wb'))
    print( 'Average accuracy for {} is:  {}'.format( modelType, np.mean(accuracies)) )  
    return model
##################################################


##################################################
# Compute accuracies across folds using an already trained model.
def validateAcrossFolds( modelType ):
    #
    accuracies = []
    for foldInd, fold in enumerate(folds):
        X_fold_test = np.take( X, fold[1], axis=0)
        y_fold_test = np.take( y, fold[1], axis=0)
        #
        if 'NN' in modelType:
            y_fold_test = to_categorical(y_fold_test)
            model = load_model( '{}/{}_fold{}.h5'.format( args.modelsPath, modelType, foldInd ) )
            accuracies.append( model.evaluate(X_fold_test, y_fold_test, batch_size=512, verbose=0 )[1] )
        else:
            model = pickle.load(open( '{}/{}_fold{}.h5'.format( args.modelsPath, modelType, foldInd ), 'rb'))
            accuracies.append( model.score(X_fold_test, y_fold_test) )
        print( '{}: {}'.format(foldInd, accuracies[-1]) )
        #
    print( 'Average accuracy for {} is:  {}'.format( modelType, np.mean(accuracies)) )  
    return model
##################################################

## Logistic Regression

In [33]:
parameters = {
    "penalty":["l2"],
    "C": [ 3., 4., 5.],
    "fit_intercept": [True],
    "class_weight":['balanced'],
    "solver":[ 'lbfgs' ],
    "multi_class": ["multinomial"],
    "random_state":[77]
    }
LR = GridSearchCV(LogisticRegression(), 
                  parameters, 
                  cv=4, 
                  n_jobs=-1)

LR = fitValidateSave( LR, 'LR' ) 

0: 0.75
{'C': 4.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
1: 0.78125
{'C': 4.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
2: 0.8125
{'C': 3.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
3: 0.8125
{'C': 3.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
4: 0.8709677419354839
{'C': 4.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
5: 0.7096774193548387
{'C': 3.0, 'class_weight': 'balanced', 'fit_intercept': True, 'multi_class': 'multinomial', 'penalty': 'l2', 'random_state': 77, 'solver': 'lbfgs'}
6: 0.8709677419354839
{'C': 3

## Random Forests

In [49]:
parameters = {
    "criterion":["gini"],
    "max_depth":[ 15, 30  ],
    "min_samples_split": [ 5 ],
    "min_samples_leaf": [1],
    "max_features":[None ],
    "random_state": [77],
    "n_estimators":[ 200 ]
    }
RF_gini = GridSearchCV(RandomForestClassifier(), 
                  parameters, 
                  cv=4, 
                  n_jobs=-1)

RF_gini = fitValidateSave( RF_gini, 'RF_gini' ) 

0: 0.8125
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
1: 0.78125
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
2: 0.84375
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
3: 0.84375
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
4: 0.9032258064516129
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
5: 0.7419354838709677
{'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 77}
6: 0.83870967741935

## Adaboost

In [8]:
from sklearn.model_selection import GridSearchCV

AB_gini = AdaBoostClassifier( base_estimator = DecisionTreeClassifier( 
                             criterion         = 'gini', 
                             splitter          = 'random',
                             max_depth         = 30, 
                             min_samples_split = 5, 
                             min_samples_leaf  = 1,
                             max_features      = None,
                             random_state      = 77 
                            ),
                            learning_rate= 1,
                            n_estimators = 200
                         )
AB_gini = fitValidateSave( AB_gini, 'AB_gini' )

0: 0.78125


AttributeError: ignored

## XGBOOST

In [57]:
XGB = XGBClassifier(  max_depth=6,  
                      learning_rate=0.1, 
                      n_estimators=100, 
                      verbosity=1, 
                      objective='multi:softmax', 
                      num_class=y_cat.shape[-1],
                      booster='gbtree', 
                      n_jobs=4, 
                      gamma=0, 
                      min_child_weight=1,
                      max_delta_step=0, 
                      subsample=.7, 
                      colsample_bytree=.6, 
                      colsample_bylevel=.6, 
                      colsample_bynode=.6, 
                      reg_alpha=.0, 
                      reg_lambda=.0, 
                      scale_pos_weight=1, 
                      base_score=0.1, 
                      random_state=77 
                      )
XGB = fitValidateSave( XGB, 'XGB' )

0: 0.78125


AttributeError: ignored

## Neural Nets (MLP)

In [0]:
##################################################
def saveToH5( data , filePath , fillvalue=0 ):
    h5f = h5py.File( filePath , 'w')
    h5f.create_dataset('dataset', data =  data ,fillvalue=fillvalue ,compression='gzip', compression_opts=4 ); 
    h5f.close()
##################################################

##################################################
def loadFromH5( filePath ):
    h5f = h5py.File( filePath , 'r')
    output =   h5f['dataset'][:]  ; h5f.close()
    return output
##################################################

##################################################
def buildMLP():
    # DEFINE MLP MODEL
    main_input = Input( shape=( X.shape[-1], ) ,  name = 'features' )
    x = Dropout(0.8) (main_input)
    #x = BatchNormalization(axis = -1)(main_input)
    x = Dense( nodes, activation='relu',
                           kernel_regularizer   =reg,
                           activity_regularizer =reg,
                           bias_regularizer     =reg
              )(x)
    x = Dropout( drops ) (x)
    
    ###
    for lay in range(layers-1):
        #
        if True:
            x = concatenate([x, main_input])
        #
        #x = BatchNormalization(axis = -1)(x)
        x = Dense( nodes, activation='relu' )(x)
        x = Dropout( drops ) (x)
    ###       
    output =  Dense( y_cat.shape[-1], activation='softmax', name = 'output' )(x)     
    
    ###
    model = Model(input=main_input, output=output)
    ###
    model.compile( optimizer=optimizer , 
                   loss='categorical_crossentropy',
                   metrics=['categorical_accuracy']
                   )
    ###
    #model.summary()  
    return model
############

############
# TRAIN MODELS
def trainMLP(modelName='NN'):
    accuracies = []
    for foldInd, fold in enumerate(folds[:]):
        model = buildMLP()
        X_fold = np.take( X, fold[0], axis=0)
        y_fold = np.take( y_cat, fold[0], axis=0)
        X_fold_test = np.take( X, fold[1], axis=0)
        y_fold_test = np.take( y_cat, fold[1], axis=0)
        
        
        loss_history =[] ;  no_improvement = 0 ; break_it = 0
        for epok in range( 0 , 1000 ) :
            if break_it :
                break
            
            model.fit(X_fold, 
                      y_fold, 
                      batch_size=batchSize,
                      shuffle=True,
                      epochs=1, 
                      verbose=0,
                      validation_data=(X_fold_test, y_fold_test)
                      )
            
            
            loss_history.append( model.evaluate( x=X_fold_test, y=y_fold_test, batch_size=512, verbose=2)[1] )
            #
            if len(loss_history)>1:
                if loss_history[-1] <= max( loss_history[:-1] ):
                    no_improvement +=1
                else:
                    no_improvement = 0
                    model.save( '{}/{}_fold{}.h5'.format( args.modelsPath, modelName, foldInd )  )
                #                            
                if no_improvement >= 20:
                    break_it = 1
                    accuracies.append( max(loss_history) )
                    print( '{}: {}'.format( foldInd, accuracies[-1] ) )
                    break
    print( 'Average accuracy for {} is:  {}'.format( 'NN', np.mean(accuracies)) )  
################################################


###########
index_ = 0 
#
layers = 2
nodes = 512
drops = 0.5
batchSize = 128
lr=0.003
optimizer = Adam(lr=lr)
reg=l1_l2(l1=0.0001, l2=0.0005)
#
trainMLP( modelName='NN{}'.format(index_) )
###########


###########
index_ = 1 
#
layers = 2
nodes = 512
drops = 0.5
batchSize = 128
lr=0.003
optimizer = Adam(lr=lr)
reg=l1_l2(l1=0.0001, l2=0.001)
#
trainMLP( modelName='NN{}'.format(index_) )
###########

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("fe..., outputs=Tensor("ou...)`


0: 0.8125
1: 0.75
2: 0.84375
3: 0.90625
4: 0.8387096524238586
5: 0.7419354915618896
6: 0.8709677457809448
7: 0.8064516186714172
8: 0.9032257795333862
9: 0.774193525314331
10: 0.8387096524238586
11: 0.8064516186714172
12: 0.6774193644523621
13: 0.9354838728904724
14: 0.8064516186714172
15: 0.774193525314331
16: 0.8387096524238586
17: 0.8709677457809448
18: 0.7096773982048035
19: 0.774193525314331
20: 0.9032257795333862
21: 0.774193525314331
22: 0.6774193644523621
23: 0.774193525314331
24: 0.8064516186714172
25: 0.8387096524238586
26: 0.9354838728904724
27: 0.8064516186714172
28: 0.7096773982048035
29: 0.8709677457809448
30: 0.8387096524238586
31: 0.8064516186714172
32: 0.9032257795333862
33: 0.8709677457809448
34: 0.7419354915618896
35: 0.8387096524238586
36: 0.8709677457809448
37: 0.774193525314331
38: 0.8387096524238586
39: 0.8387096524238586
Average accuracy for NN is:  0.8174899101257325
0: 0.8125
1: 0.75
2: 0.8125
3: 0.90625
4: 0.8709677457809448
5: 0.774193525314331
6: 0.870967745

## Some more functions to generate the Ensemble prediction on Test dataset

In [0]:
##################################################
def genTestPredictionsPerModelInstance( modelInstance ):
    # Generate predictions over test set
    predictions = np.zeros(( testData.shape[0], y_cat.shape[-1] ))
    for foldIndex, fold in enumerate(folds[:]):
        print(foldIndex)
        if 'NN' in modelInstance:
            model = load_model( '{}/{}_fold{}.h5'.format( args.modelsPath, modelInstance, foldIndex ) )
            predictions += model.predict(testData, batch_size=1024) 
        else:
            model = pickle.load(open( '{}/{}_fold{}.h5'.format( args.modelsPath, modelInstance, foldIndex ), 'rb'))
            predictions += model.predict_proba(testData)
    #
    predictionsPath = '{}/predictions_{}.h5'.format( args.resultsPath, modelInstance )
    saveToH5( predictions, predictionsPath ) 
    return model
##################################################


##################################################
# Get predictions 
def generateEnsemblePredictions(  ensemble = [ 'LR', 'RF_gini', 'AB_gini', 'NN', 'GB' ],                   
                                  mode='sum'
                                ): 
    #                      
    ensemblePredictions = np.zeros(( testData.shape[0], len(ensemble), y_cat.shape[-1] ))
    for modelInstanceIndex, modelInstance in enumerate(ensemble) :
        print(modelInstance)
        predictionsPath  = '{}/predictions_{}.h5'.format( args.resultsPath, modelInstance ) 
        modelPredictions = loadFromH5(predictionsPath)
        ensemblePredictions[:,modelInstanceIndex] += np.copy( modelPredictions ) 
    #
    if mode=='sum':
        classPredictions = np.sum(ensemblePredictions, axis=1)   
    elif mode=='max':
        classPredictions = np.max(ensemblePredictions, axis=1)   
    
    #
    classPredictions = np.argmax( classPredictions, axis=1 )      
    predictions = testData.copy()
    predictions['predictions'] = classPredictions 
    predictions = predictions['predictions']
    predictions.to_csv( '{}/v0.1'.format(args.resultsPath ), index_label='ID', header=['label']   )
##################################################

## Build Ensemble predictions

In [2]:

## Generate test predictions per modelInstance
for modelInstance in [ 'LR', 'RF_gini', 'AB_gini', 'NN0', 'NN1'  ]:
    print(modelInstance)
    genTestPredictionsPerModelInstance( modelInstance ) 

LR


NameError: ignored

In [0]:
# Generate Ensemble Predictions
generateEnsemblePredictions( ensemble = [ 'LR', 'RF_gini', 'AB_gini', 'NN0', 'NN1' ] 
                           )